# The Basis Mixer!

The previous notebooks explored the individual components of the Basis Function Models. In this notebook we will put everything together to render an expressive performance of a piece given its score.

We start by 

In [ ]:
from partitura import save_performance_midi
from basismixer.predictive_models import FullPredictiveModeldictiveModel

In [ ]:
# Load model
model = FullPredictiveModel(model_config)

# make basis functions
part = load_musicxml(musicxml_fn, force_note_ids=True)
onsets = part.note_array['onset']
basis_functions = make_basis_functions(input_musicxml)
# Predict performance
predicted_ppart = model.predict(part, onsets)





Finally, we can export the predictions of the model as a MIDI file!

In [ ]:
# export midi
midi_fn = 'my_midi_output.mid'
save_performance_midi(predicted_ppart, midi_fn)